In [5]:
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image
import winsound
import threading
import time

cap = cv2.VideoCapture(0)

# Load models
yawn_model = tf.keras.models.load_model('model/ENB_yawn.h5', compile=False)
eye_model = tf.keras.models.load_model('model/ENB.h5', compile=False)

# Load cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'cascade/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'cascade/haarcascade_eye.xml')

# Global Variables for State Tracking
yawn_counter = 0
CLOSED_EYES_COUNTER = 0
ALERT_LEVEL = 0
YAWN_THRESHOLD = 5
EYE_CLOSURE_THRESHOLD = 3
last_yawn_time = None
last_blink_time = None
RESET_TIME = 2  # Time in seconds to reset the counter

# Function to update alert level
def update_alert_level():
    global ALERT_LEVEL
    if yawn_counter >= YAWN_THRESHOLD or CLOSED_EYES_COUNTER >= EYE_CLOSURE_THRESHOLD:
        ALERT_LEVEL += 1
    else:
        ALERT_LEVEL = max(ALERT_LEVEL - 1, 0)

# Function to trigger alerts
def trigger_alert():
    duration = 500  # Duration of the beep in milliseconds
    if ALERT_LEVEL == 1:
        frequency = 500  # Lower frequency for a softer alert
        winsound.Beep(frequency, duration)
    elif ALERT_LEVEL == 2:
        frequency = 1000  # Moderate frequency for a more noticeable alert
        winsound.Beep(frequency, duration)
    elif ALERT_LEVEL >= 3:
        frequency = 2000  # High frequency for an urgent alert
        winsound.Beep(frequency, duration)

# Function to predict yawn or no_yawn
def predict_face(face_roi, frame, x, y, w, h):
    global yawn_counter, last_yawn_time
    face_img = cv2.resize(face_roi, (224, 224))
    face_img = tf.keras.preprocessing.image.img_to_array(face_img)
    face_img = tf.expand_dims(face_img, 0)
    predictions = yawn_model.predict(face_img)
    class_labels = ['No Yawn', 'Yawn']
    score = tf.nn.softmax(predictions[0])
    prediction = class_labels[tf.argmax(score)]

    if prediction == 'Yawn':
        yawn_counter += 1
        last_yawn_time = time.time()
    else:
        if last_yawn_time and (time.time() - last_yawn_time > RESET_TIME):
            yawn_counter = 0
            last_yawn_time = None

    cv2.putText(frame, prediction, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Function to predict open or closed eyes
def predict_eye(eye_region, roi_color, ex, ey, ew, eh):
    global CLOSED_EYES_COUNTER, eye_closure_counter, last_blink_time
    img = Image.fromarray(eye_region).resize((224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = eye_model.predict(img_array)
    class_labels = ['close eyes', 'open eyes']
    score = tf.nn.softmax(predictions[0])
    label = class_labels[tf.argmax(score)]

    if label == 'close eyes':
        CLOSED_EYES_COUNTER += 1
        last_blink_time = time.time()
    else:
        if last_blink_time and (time.time() - last_blink_time > RESET_TIME):
            CLOSED_EYES_COUNTER = 0
            last_blink_time = None

    cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
    cv2.putText(roi_color, label, (ex, ey - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)



while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    threads = []  # List to collect all threads

    for (x, y, w, h) in faces:
        face_thread = threading.Thread(target=predict_face, args=(frame[y:y + h, x:x + w], frame, x, y, w, h))
        face_thread.start()
        threads.append(face_thread)

        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            eye_thread = threading.Thread(target=predict_eye, args=(roi_color[ey:ey + eh, ex:ex + ew], roi_color, ex, ey, ew, eh))
            eye_thread.start()
            threads.append(eye_thread)

    for thread in threads:
        thread.join()

    update_alert_level()
    trigger_alert()

    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
